# YOLOv8 Seal Face Detector

## Install Dependencies

In [ ]:
%%capture
!pip install -U wandb roboflow ultralytics torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117 --user


In [ ]:
# Check whether GPU is available
import torch
torch.cuda.is_available()

## Initialize Weights and Biases

In [ ]:
!wandb login # login to wandb

In [ ]:
import os
import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = "sealnet-yolov8-unified.ipynb"
    

## Import Roboflow data 
You can find the Roboflow data [here](https://app.roboflow.com/fruitpunch-ai-private-workspace/seal-face-detection/1#).

In [ ]:
from roboflow import Roboflow

ROBOFLOW_WORKSPACE = "fruitpunch-ai-private-workspace"
ROBOFLOW_PROJECT = "colgate-harbour-seal-dataset"
ROBOFLOW_VERSION = 8

with open(file="./roboflow.apikey", mode="r") as f:
  ROBOFLOW_API_KEY = f.readline().strip()

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
dataset = project.version(ROBOFLOW_VERSION).download("yolov8", location="datasets/unified-seal-facial-detection-colgate-fur-seals-3", overwrite=False)

In [ ]:
dataset.location


In [ ]:
with open(f"{dataset.location}/data.yaml", 'r') as f:
    print(f.read())

## Train the Yolov8 Model

### Create and register callbacks to send model evaluation metrics to Weights & Biases 

In [ ]:
from ultralytics import YOLO
from ultralytics.yolo.utils.torch_utils import get_flops, get_num_params


def on_pretrain_routine_start(trainer):
    run = wandb.init(
        name=trainer.args.name,
        project=trainer.args.project or "YOLOv8",
        tags=["YOLOv8"],
        config=vars(trainer.args),
        resume="allow",
    )

    assert wandb.run is not None
    wandb.run.log_code(include_fn=lambda path: path.endswith(".ipynb"))

def on_train_epoch_start(trainer):
    wandb.log({
        "epoch": trainer.epoch + 1,
    }, step=trainer.epoch + 1)

def on_train_epoch_end(trainer):
    wandb.log({
        **trainer.metrics,
        **trainer.label_loss_items(trainer.tloss, prefix="train"),
        **({
            "train_batch_images": [
                wandb.Image(str(image_path), caption=image_path.stem)
                for image_path in trainer.save_dir.glob("train_batch*.jpg")
            ]
        } if trainer.epoch == 1 else {}),
    }, step=trainer.epoch + 1)


def on_fit_epoch_end(trainer):
    wandb.log({
        **trainer.metrics,
        "model/parameters": get_num_params(trainer.model),
        "model/GFLOPs": round(get_flops(trainer.model), 3),
        "model/speed(ms)": round(trainer.validator.speed[1], 3),
    }, step=trainer.epoch + 1)

def on_train_end(trainer):
    
    wandb.log(
        {
            "results": [
                wandb.Image(str(image_path), caption=image_path.stem)
                for image_path in trainer.save_dir.glob("*.png")
            ]
        },
        step=trainer.epoch + 1
    )


def teardown(trainer):
    wandb.finish()


callbacks = (
    {
        "on_pretrain_routine_start": on_pretrain_routine_start,
        "on_train_epoch_start": on_train_epoch_start,
        "on_train_epoch_end": on_train_epoch_end,
        "on_fit_epoch_end": on_fit_epoch_end,
        "on_train_end": on_train_end,
        "teardown": teardown,
    }
    if wandb
    else {}
)

for event, func in callbacks.items():
    YOLO.add_callback(event, func)


In [ ]:
from ultralytics import YOLO

# Load a model
configs = [{
    "epochs": 15,
    "model": "yolov8n"
}, {
    "epochs": 30,
    "model": "yolov8s"
}, {
    "epochs": 45,
    "model": "yolov8s"
},
{
    "epochs": 15,
    "model": "yolov8m"
}, {
    "epochs": 30,
    "model": "yolov8m"
}]

for config in configs:

    model = YOLO(f"{config['model']}.pt")  # load a pretrained model (recommended for training)

    # Use the model
    results = model.train(data=f"{dataset.location}/data.yaml", epochs=config['epochs'], project="sealnet", name=f"aideenfay-unified-{config['model']}-{config['epochs']}epochs")  # train the model



## Run the model on the test dataset

In [ ]:
test_results = model.val(data=f"{dataset.location}/data.yaml", project="sealnet", name=f"aideenfay-unified-{config['model']}-{config['epochs']}epochs", split="test", save=True)